Connected to Python 3.13.7

In [ ]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping


def build_finetune_model(num_classes, input_shape=(224,224,3), base_trainable=False):
    base = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    for layer in base.layers[:-30]:
        layer.trainable = False
    for layer in base.layers[-30:]:
        layer.trainable = True
    x = base.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)
    out = Dense(num_classes, activation='softmax')(x)
    return Model(inputs=base.input, outputs=out)


def main(data_dir, out_path=r'api\src\train_cnn.ipynb', batch_size=32, epochs=15):
    # Use ImageDataGenerator with validation split
    datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
    )
    train_gen = datagen.flow_from_directory(data_dir, target_size=(224,224), batch_size=batch_size, subset='training')
    val_gen = datagen.flow_from_directory(data_dir, target_size=(224,224), batch_size=batch_size, subset='validation')

    num_classes = train_gen.num_classes
    model = build_finetune_model(num_classes)
    model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

    callbacks = [
        ModelCheckpoint(out_path, monitor='val_accuracy', save_best_only=True, verbose=1),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1),
        EarlyStopping(monitor='val_loss', patience=7, verbose=1, restore_best_weights=True)
    ]

    model.fit(train_gen, validation_data=val_gen, epochs=epochs, callbacks=callbacks)
    model.save(out_path.replace('.h5','.keras'))
    print('Saved', out_path)


if __name__ == '__main__':



    DEFAULT_DATA_PATH = r"C:\Users\USER\Documents\AI\computervision\data\maize"

    main(DEFAULT_DATA_PATH, epochs=20)

Found 2349 images belonging to 4 classes.
Found 585 images belonging to 4 classes.


c:\Users\USER\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4744 - loss: 1.1491
Epoch 1: val_accuracy improved from None to 0.38462, saving model to models/cnn_model.h5


74/74 ━━━━━━━━━━━━━━━━━━━━ 142s 2s/step - accuracy: 0.5636 - loss: 1.0072 - val_accuracy: 0.3846 - val_loss: 1.2384 - learning_rate: 1.0000e-04
Epoch 2/20
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6661 - loss: 0.8196
Epoch 2: val_accuracy improved from 0.38462 to 0.61709, saving model to models/cnn_model.h5


74/74 ━━━━━━━━━━━━━━━━━━━━ 147s 2s/step - accuracy: 0.6931 - loss: 0.7584 - val_accuracy: 0.6171 - val_loss: 0.8634 - learning_rate: 1.0000e-04
Epoch 3/20
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6964 - loss: 0.7619
Epoch 3: val_accuracy did not improve from 0.61709
74/74 ━━━━━━━━━━━━━━━━━━━━ 165s 2s/step - accuracy: 0.6935 - loss: 0.7501 - val_accuracy: 0.3043 - val_loss: 2.5818 - learning_rate: 1.0000e-04
Epoch 4/20
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7436 - loss: 0.6214
Epoch 4: val_accuracy did not improve from 0.61709
74/74 ━━━━━━━━━━━━━━━━━━━━ 490s 7s/step - accuracy: 0.7378 - loss: 0.6372 - val_accuracy: 0.2615 - val_loss: 5.3611 - learning_rate: 1.0000e-04
Epoch 5/20
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7553 - loss: 0.6122
Epoch 5: val_accuracy did not improve from 0.61709

Epoch 5: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.
74/74 ━━━━━━━━━━━━━━━━━━━━ 145s 2s/step - accuracy: 0.7471 - loss: 0.6303 - val_accuracy:


Epoch 8: ReduceLROnPlateau reducing learning rate to 2.499999936844688e-05.
74/74 ━━━━━━━━━━━━━━━━━━━━ 180s 2s/step - accuracy: 0.7820 - loss: 0.5375 - val_accuracy: 0.7248 - val_loss: 1.1078 - learning_rate: 5.0000e-05
Epoch 9/20
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.7893 - loss: 0.4835
Epoch 9: val_accuracy did not improve from 0.72479
74/74 ━━━━━━━━━━━━━━━━━━━━ 245s 3s/step - accuracy: 0.8054 - loss: 0.4762 - val_accuracy: 0.6530 - val_loss: 0.8069 - learning_rate: 2.5000e-05
Epoch 10/20
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8007 - loss: 0.4868
Epoch 10: val_accuracy improved from 0.72479 to 0.72821, saving model to models/cnn_model.h5


74/74 ━━━━━━━━━━━━━━━━━━━━ 181s 2s/step - accuracy: 0.7999 - loss: 0.4813 - val_accuracy: 0.7282 - val_loss: 0.6076 - learning_rate: 2.5000e-05
Epoch 11/20
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.8031 - loss: 0.4690
Epoch 11: val_accuracy did not improve from 0.72821
74/74 ━━━━━━━━━━━━━━━━━━━━ 260s 3s/step - accuracy: 0.8089 - loss: 0.4679 - val_accuracy: 0.6872 - val_loss: 0.7794 - learning_rate: 2.5000e-05
Epoch 12/20
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8137 - loss: 0.4434
Epoch 12: val_accuracy did not improve from 0.72821
74/74 ━━━━━━━━━━━━━━━━━━━━ 179s 2s/step - accuracy: 0.8084 - loss: 0.4589 - val_accuracy: 0.5453 - val_loss: 1.8481 - learning_rate: 2.5000e-05
Epoch 13/20
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8270 - loss: 0.4414
Epoch 13: val_accuracy did not improve from 0.72821

Epoch 13: ReduceLROnPlateau reducing learning rate to 1.249999968422344e-05.
74/74 ━━━━━━━━━━━━━━━━━━━━ 235s 3s/step - accuracy: 0.8131 - loss: 0.4540 - val_ac

74/74 ━━━━━━━━━━━━━━━━━━━━ 204s 3s/step - accuracy: 0.8178 - loss: 0.4400 - val_accuracy: 0.8154 - val_loss: 0.4152 - learning_rate: 1.2500e-05
Epoch 15/20
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.8241 - loss: 0.4399
Epoch 15: val_accuracy did not improve from 0.81538
74/74 ━━━━━━━━━━━━━━━━━━━━ 230s 3s/step - accuracy: 0.8199 - loss: 0.4514 - val_accuracy: 0.8068 - val_loss: 0.4123 - learning_rate: 1.2500e-05
Epoch 16/20
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8310 - loss: 0.4570
Epoch 16: val_accuracy did not improve from 0.81538
74/74 ━━━━━━━━━━━━━━━━━━━━ 180s 2s/step - accuracy: 0.8310 - loss: 0.4358 - val_accuracy: 0.7624 - val_loss: 0.5688 - learning_rate: 1.2500e-05
Epoch 17/20
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8306 - loss: 0.4204
Epoch 17: val_accuracy improved from 0.81538 to 0.84444, saving model to models/cnn_model.h5


74/74 ━━━━━━━━━━━━━━━━━━━━ 179s 2s/step - accuracy: 0.8208 - loss: 0.4332 - val_accuracy: 0.8444 - val_loss: 0.4045 - learning_rate: 1.2500e-05
Epoch 18/20
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8336 - loss: 0.4062
Epoch 18: val_accuracy did not improve from 0.84444
74/74 ━━━━━━━━━━━━━━━━━━━━ 180s 2s/step - accuracy: 0.8276 - loss: 0.4151 - val_accuracy: 0.6615 - val_loss: 0.9605 - learning_rate: 1.2500e-05
Epoch 19/20
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8323 - loss: 0.3945
Epoch 19: val_accuracy did not improve from 0.84444
74/74 ━━━━━━━━━━━━━━━━━━━━ 179s 2s/step - accuracy: 0.8327 - loss: 0.4015 - val_accuracy: 0.6085 - val_loss: 1.6942 - learning_rate: 1.2500e-05
Epoch 20/20
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8318 - loss: 0.4140
Epoch 20: val_accuracy improved from 0.84444 to 0.84786, saving model to models/cnn_model.h5


74/74 ━━━━━━━━━━━━━━━━━━━━ 183s 2s/step - accuracy: 0.8242 - loss: 0.4306 - val_accuracy: 0.8479 - val_loss: 0.3398 - learning_rate: 1.2500e-05
Restoring model weights from the end of the best epoch: 20.
Restoring model weights from the end of the best epoch: 20.
Saved models/cnn_model.h5
